In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely
import matplotlib.pyplot as plt
import seaborn as sns

#from geopandas import GeoDataFrame
from shapely.geometry import Point,Polygon, LineString,shape

from shapely.strtree import STRtree
from shapely.ops import cascaded_union,unary_union

import os
from tqdm import tqdm # Para barra de progreso mas Cool


In [2]:
# User Functions
#==============================================================
def Get_Files(path):
    xfile = []
    try:
        for files in tqdm(os.listdir(path)):
            file = files.split('.')
            if file[1]=='shp':
                xfile.append(files)
    except:
        print('error en el archivo {}'.format(path))
    return xfile

# Cargar Datos extras de excel
#===============================================================
def get_extras(path, file):
    return pd.read_excel(path+file)

# Cargar Datos de los Yields
#===============================================================
def get_yield(path,file,var):
    yields = gpd.read_file(path+file)
    return yields[var].copy(),yields
    
# Agregar las variables ecxtras
#===============================================================
def get_vars(yields,extras,yield_):
    lote = yields.loc[1,'Lote'] # Filtra Lote
    date = yield_.loc[1,'Tiempo'] # Obtiene la fecha del archivo original
    date = date.split('-')[0]
    tipo = extras[extras['Lotes']==lote].copy()
    tipo['cam']= tipo['campaña'].apply(lambda x: x.split('-')[0]) # Agrego solo año inicial campaña
    tp = tipo[tipo['cam']==date].copy()
    
    # Agrego Variables Extras
    #==============================================================
    yields['Cultivo']=list(tp['cultivo'].values)[0]
    yields['Genotipo']=list(tp['genotipo'].values)[0]
    yields['campana']=list(tp['campaña'].values)[0]
    yields['fecha_s']=list(tp['fecha de siembra'].values)[0]
    
    return yields   

# Carga de los archivos

Cargaremos los datos de la cosecha según los archivos disponibles.

In [3]:
# Inidicamos el Path del directorio con archivos Yield y los datos de variables extras
#==============================================================================================
path = '../Data/yield_BL1/'
pathE = '../Data/Extras/'


Obtener nombre los archivos disponibles en le path indicado.

In [5]:
# Buscar Archivos del Yield
#==================================================
files = Get_Files(path)
files

100%|██████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:00<?, ?it/s]


['yield_mz2_2018.shp',
 'yield_mz_2015.shp',
 'yield_mz_2017.shp',
 'yield_sy_2016.shp',
 'yield_sy_2019.shp',
 'yield_wh_2018.shp']

# Generar el dataframe

Cargamos las variables extras disponibles en un archivo excel.

In [6]:
# Load variables extras
#=============================================================
extras = get_extras(pathE, 'bd_campaña_cultivo_genotipo_fs.xlsx')   

Ahora cargamos los datos de los yields y los combinamos con las varaibles extras.

In [7]:
# Las variables de inters del Yiled
#===================================================================
var = ['Lote','_seco_Masa','geometry']

all_fields =  pd.DataFrame()

for file in tqdm(files):
    
    yields,yields_ = get_yield(path,file,var)
    temp = get_vars(yields,extras,yields_)
    
    all_fields = pd.concat([all_fields,temp],axis=0)
    
    
all_fields.reset_index(drop = True, inplace=True)

100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [01:19<00:00, 13.26s/it]


In [9]:
all_fields.head()

,Lote,_seco_Masa,geometry,Cultivo,Genotipo,campana,fecha_s
0,BL1,1.1129,POINT (-61.58942 -34.38390),tr,Ceibo,2018-2019,2022-07-02
1,BL1,1.8112,POINT (-61.58941 -34.38390),tr,Ceibo,2018-2019,2022-07-02
2,BL1,2.0609,POINT (-61.58941 -34.38391),tr,Ceibo,2018-2019,2022-07-02
3,BL1,1.5176,POINT (-61.58940 -34.38392),tr,Ceibo,2018-2019,2022-07-02
4,BL1,1.1957,POINT (-61.58939 -34.38393),tr,Ceibo,2018-2019,2022-07-02


# Escribir Dataframe en archivo csv

In [10]:
all_fields.to_csv('Yields.csv',index=False)